# 0.0. IMPORTS

## 0.1. Bibliotecas

In [86]:
import pandas as pd
import numpy as np
import os

from unidecode import unidecode

## 0.2. Carregar Dados

### 0.2.1. Escolas

In [87]:
diretorio_escolas = os.path.dirname(os.path.abspath('escolas.csv'))
caminho_escolas = os.path.join(diretorio_escolas, '../..', 'data', 'raw', 'escolas.csv')
df_escolas_raw = pd.read_csv(caminho_escolas)

### 0.2.2. Material Didatico

In [88]:
diretorio_material_didatico = os.path.dirname(os.path.abspath('material_didatico.csv'))
caminho_material_didatico = os.path.join(diretorio_material_didatico, '../..', 'data', 'raw', 'material_didatico.csv')
df_material_didatico_raw = pd.read_csv(caminho_material_didatico)

### 0.2.3. Sub-Prefeituras

In [89]:
diretorio_subprefeituras = os.path.dirname(os.path.abspath('subprefeituras.csv'))
caminho_subprefeituras = os.path.join(diretorio_subprefeituras, '../..', 'data', 'raw', 'subprefeituras.csv')
df_subprefeituras_raw = pd.read_csv(caminho_subprefeituras)

## 0.3. Funções

### 0.3.1. renomeia_colunas

In [90]:
def renomeia_colunas(data):
    return [x.lower() for x in data.columns]

### 0.3.2. tamanho_dos_dados

In [91]:
def tamanho_dos_dados(data):
    print(f'Colunas: {data.shape[1]}')
    print(f'Linhas: {data.shape[0]}')

### 0.3.3. qtd_total_unicos_por_coluna

In [92]:
def qtd_total_unicos_por_coluna(data):
    for x in data.columns:
        print(f'Quantidade de valores unicos em {x}: {len(data[x].unique())}')

### 0.3.4. get_numericos

In [93]:
def get_numericos(data):
    return data.select_dtypes(include=['int32', 'int64', 'float32', 'float64'])

### 0.3.5. get_categoricos

In [94]:
def get_categoricos(data):
    return data.select_dtypes(exclude=['int32', 'int64', 'float32', 'float64', 'datetime64[ns]'])

### 0.3.6. get_descriptive_info

In [95]:
def get_descriptive_info(data):
    dados_numericos = get_numericos(data)
    
    # medidas de tendencia central - media, mediana
    df_media = pd.DataFrame(dados_numericos.apply(np.mean)).T
    df_mediana = pd.DataFrame(dados_numericos.apply(np.median)).T   
    
    # medidas de dispersão - min, max, range, skew, kurtosis, std
    df_std = pd.DataFrame(dados_numericos.apply(np.std)).T
    df_min = pd.DataFrame(dados_numericos.apply(min)).T
    df_max = pd.DataFrame(dados_numericos.apply(max)).T
    df_range = pd.DataFrame(dados_numericos.apply(lambda x: x.max() - x.min())).T
    df_skew = pd.DataFrame(dados_numericos.apply(lambda x: x.skew())).T
    df_kurtosis = pd.DataFrame(dados_numericos.apply(lambda x: x.kurtosis())).T
    
    #min, max, range, media, mediana, std, skew, kurtosis
    df_descriptive_info = pd.concat([df_min, df_max, df_range,
                                     df_media, df_mediana, df_std,
                                     df_skew, df_kurtosis]).T.reset_index()
    
    df_descriptive_info.columns = ['attributes', 'min', 'max',
                                   'range', 'media', 'mediana',
                                   'std', 'skew', 'kurtosis']
    
    return df_descriptive_info

### 0.3.7. salvar_dados_csv

In [96]:
def salvar_dados_csv(data, nome=''):
    data.to_csv(f'../../data/processed/{nome}', index=False )

### 0.3.8. remove_acentos

In [97]:
def remove_acentos(data):
    categoricos = get_categoricos(data)
    for column in categoricos.columns:        
        data[column] = data[column].apply(lambda x: unidecode(x))
    return data

### 0.3.9. string_maisculo

In [98]:
def string_maisculo(data):
    categoricos = get_categoricos(data)

    for column in categoricos:
        data[column] = data[column].str.upper()
        
    return data

### 0.3.10. remodelando_id

In [99]:
def remodelando_id(data):
    data['id'] = data['id'].astype(str).apply(lambda x: f'0{x}' if len(x) < 3 else f'{x}')
    return data

# 1.0. DESCRIÇÃO DOS DADOS

## 1.1. Escolas

### 1.1.1. Visão Geral dos Dados

In [100]:
df1_escolas = df_escolas_raw.copy()

In [101]:
df1_escolas.head().T

,0,1,2,3,4
id,178,634,483,476,132
Escolas_Postos,CENTRO INTEGRADO DE EDUCAÇÃO PÚBLICA HENFIL,EM ALICE DO AMARAL PEIXOTO,EM CELESTINO SILVA,ESCOLA MUNICIPAL FLORIANO PEIXOTO,EM PEREIRA PASSOS
BAIRRO,CAJU,BENFICA,CENTRO,SÃO CRISTÓVÃO,RIO COMPRIDO
ENDEREÇO,Rua Carlos Seidl S/Nº,Rua Ébano 187,"R. do Lavradio, 56",Praça Argentina 20,Praça Condessa Paulo de Frontin 45
lat,"-22,880888","-22,889574","-22,909293","-22,897629","-22,924412"
lon,"-43,225326","-43,236202","-43,183579","-43,227456","-43,208579"


In [102]:
df1_escolas.columns = renomeia_colunas(df1_escolas)

In [103]:
df1_escolas.columns

Index(['id', 'escolas_postos', 'bairro', 'endereço ', 'lat', 'lon'], dtype='object')

### 1.1.2. Tamanho dos dados

In [104]:
tamanho_dos_dados(df1_escolas)

Colunas: 6
Linhas: 152


In [105]:
qtd_total_unicos_por_coluna(df1_escolas)

Quantidade de valores unicos em id: 136
Quantidade de valores unicos em escolas_postos: 136
Quantidade de valores unicos em bairro: 90
Quantidade de valores unicos em endereço : 137
Quantidade de valores unicos em lat: 136
Quantidade de valores unicos em lon: 135


### 1.1.3. Verificar Nulos

In [106]:
df1_escolas.isna().sum()

id                0
escolas_postos    0
bairro            0
endereço          0
lat               0
lon               0
dtype: int64

### 1.1.4. Verificar tipos dos dados

In [107]:
df1_escolas.dtypes

id                 int64
escolas_postos    object
bairro            object
endereço          object
lat               object
lon               object
dtype: object

### 1.1.5. Verificar dados duplicados

In [108]:
df1_escolas.duplicated().sum()

13

### 1.1.6. Tratamento Inicial

In [109]:
df1_escolas = df1_escolas.drop_duplicates()

In [110]:
df1_escolas

,id,escolas_postos,bairro,endereço,lat,lon
0,178,CENTRO INTEGRADO DE EDUCAÇÃO PÚBLICA HENFIL,CAJU,Rua Carlos Seidl S/Nº,"-22,880888","-43,225326"
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,Rua Ébano 187,"-22,889574","-43,236202"
2,483,EM CELESTINO SILVA,CENTRO,"R. do Lavradio, 56","-22,909293","-43,183579"
3,476,ESCOLA MUNICIPAL FLORIANO PEIXOTO,SÃO CRISTÓVÃO,Praça Argentina 20,"-22,897629","-43,227456"
4,132,EM PEREIRA PASSOS,RIO COMPRIDO,Praça Condessa Paulo de Frontin 45,"-22,924412","-43,208579"
...,...,...,...,...,...,...
146,210,EM DEBORA MENDES DE MORAES,PEDRA DE GUARATIBA,Rua Belchior Da Fonseca 643,"-23,001416","-43,638757"
147,474,EM ENGENHEIRO GASTÃO RANGEL,GUARATIBA,Estrada Do Magarça 9.183,"-22,98046","-43,643545"
148,301,EM JONATAS SERRANO,GUARATIBA,"Estrada Do Mato Alto, S/Nº","-22,953163","-43,577409"
149,215,E.M. NARCISA AMALIA,ILHA DE GUARATIBA,Estrada Teodoreto de Camargo S/N.º,"-23,009084","-43,537582"


In [111]:
df1_escolas['endereço '] = df1_escolas['endereço '].str.replace("R.", "Rua")

C:\Users\jessi\AppData\Local\Temp\ipykernel_23916\3556405822.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df1_escolas['endereço '] = df1_escolas['endereço '].str.replace("R.", "Rua")


In [112]:
df1_escolas['lat'] = df1_escolas['lat'].str.replace(',', '.')
df1_escolas['lat'] = pd.to_numeric(df1_escolas['lat']).round(5)

df1_escolas['lon'] = df1_escolas['lon'].str.replace(',', '.')
df1_escolas['lon'] = pd.to_numeric(df1_escolas['lon']).round(5)

df1_escolas.dtypes

id                  int64
escolas_postos     object
bairro             object
endereço           object
lat               float64
lon               float64
dtype: object

In [113]:
df1_escolas = remove_acentos(df1_escolas)

In [114]:
df1_escolas = remodelando_id(df1_escolas)

In [115]:
df1_escolas = string_maisculo(df1_escolas)

In [116]:
tamanho_dos_dados(df1_escolas)

Colunas: 6
Linhas: 139


### 1.1.6. Salvar dados

In [117]:
salvar_dados_csv(df1_escolas, 'df1_escolas_descricao_dados.csv')

## 1.2. Material Didatico

### 1.2.1. Visão Geral dos Dados

In [118]:
df1_material_didatico = df_material_didatico_raw.copy()

In [119]:
df1_material_didatico.head().T

,0,1,2,3,4
id,178,634,483,476,132
Quantidade,20,121,220,190,67


In [120]:
df1_material_didatico.columns = renomeia_colunas(df1_material_didatico)

### 1.2.2. Tamanho dos dados

In [121]:
tamanho_dos_dados(df1_material_didatico)

Colunas: 2
Linhas: 151


In [122]:
qtd_total_unicos_por_coluna(df1_material_didatico)

Quantidade de valores unicos em id: 151
Quantidade de valores unicos em quantidade: 135


### 1.2.3. Verificar Nulos

In [123]:
df1_material_didatico.isna().sum()

id            0
quantidade    4
dtype: int64

### 1.2.4. Verificar tipos dos dados

In [124]:
df1_material_didatico.dtypes

id             int64
quantidade    object
dtype: object

### 1.2.5. Verificar dados duplicados

In [125]:
df1_material_didatico.duplicated().sum()

0

### 1.2.6. Tratamento Inicial

In [126]:
df1_material_didatico['quantidade'].unique()

array(['20', '121', '220', '190', '67', '397', '92', '578', '591', '473',
       '456', '223', '420', '573', "''", '366', '516', '319', '308',
       '111', '48', '525', '445', '164', '293', '152', '399', '185',
       '576', '63', '491', "'NaN'", '336', '159', '342', '262', '440',
       '365', nan, '183', '297', '44', '109', '350', '255', '457', '493',
       '489', '64', '530', '158', '123', '468', '234', '24', '218', '151',
       '564', '504', '61', '426', '167', '509', '292', '210', '153',
       "'397'", '41', '561', '180', '257', '260', '317', '345', '32',
       '513', '338', '416', '352', '393', '425', '136', "'395'", '115',
       '534', '430', '466', '51', '26', '94', '128', '520', '524', '279',
       '161', '179', '194', '599', '315', '435', '400', '294', '148',
       '269', '571', '235', '187', '392', '362', '71', '221', '372',
       '479', '99', '157', '296', '330', '316', '106', '207', '459',
       '537', '162', '579', '517', "' '", '135', '118', '341', '329',
     

In [127]:
df1_material_didatico['quantidade'] = df1_material_didatico['quantidade'].str.replace("'", "")
df1_material_didatico['quantidade'] = df1_material_didatico['quantidade'].replace(" ", np.nan)
df1_material_didatico['quantidade'] = df1_material_didatico['quantidade'].replace('NaN', np.nan)

# Converter a coluna 'quantidade' para tipo numérico
df1_material_didatico['quantidade'] = pd.to_numeric(df1_material_didatico['quantidade'])

moda = df1_material_didatico['quantidade'].value_counts().idxmax()
df1_material_didatico['quantidade'] = df1_material_didatico['quantidade'].fillna(moda)


# Verificar se as correções foram aplicadas corretamente
print(df1_material_didatico['quantidade'].unique())


[ 20. 121. 220. 190.  67. 397.  92. 578. 591. 473. 456. 223. 420. 573.
 342. 366. 516. 319. 308. 111.  48. 525. 445. 164. 293. 152. 399. 185.
 576.  63. 491. 336. 159. 262. 440. 365. 183. 297.  44. 109. 350. 255.
 457. 493. 489.  64. 530. 158. 123. 468. 234.  24. 218. 151. 564. 504.
  61. 426. 167. 509. 292. 210. 153.  41. 561. 180. 257. 260. 317. 345.
  32. 513. 338. 416. 352. 393. 425. 136. 395. 115. 534. 430. 466.  51.
  26.  94. 128. 520. 524. 279. 161. 179. 194. 599. 315. 435. 400. 294.
 148. 269. 571. 235. 187. 392. 362.  71. 221. 372. 479.  99. 157. 296.
 330. 316. 106. 207. 459. 537. 162. 579. 517. 135. 118. 341. 329. 570.
 320. 335. 521. 225.]


In [128]:
df1_material_didatico = remodelando_id(df1_material_didatico)
df1_material_didatico['id'].unique()

array(['178', '634', '483', '476', '132', '410', '055', '490', '600',
       '089', '387', '331', '450', '558', '413', '698', '540', '309',
       '268', '523', '017', '035', '011', '226', '638', '271', '126',
       '180', '163', '361', '280', '060', '587', '653', '660', '382',
       '217', '378', '429', '025', '367', '373', '624', '157', '020',
       '117', '502', '351', '135', '647', '440', '259', '422', '041',
       '537', '230', '683', '146', '566', '388', '498', '663', '121',
       '161', '263', '118', '140', '260', '236', '090', '274', '676',
       '390', '608', '589', '256', '091', '285', '248', '441', '623',
       '603', '673', '182', '520', '412', '031', '514', '023', '415',
       '016', '570', '528', '414', '690', '202', '550', '211', '526',
       '492', '466', '636', '527', '586', '080', '398', '175', '308',
       '512', '576', '495', '379', '641', '689', '171', '477', '206',
       '687', '568', '399', '255', '262', '633', '616', '320', '144',
       '534', '342',

### 1.2.7. Salvar dados

In [129]:
salvar_dados_csv(df1_material_didatico, 'df1_material_didatico_descricao_dados.csv')

## 1.3. Sub-Prefeituras

### 1.3.1. Visão Geral dos Dados

In [130]:
df1_subprefeituras = df_subprefeituras_raw.copy()

In [131]:
df1_subprefeituras.head().T

,0,1,2,3,4
nome,Santo Cristo,Gamboa,Saúde,Caju,Centro
subprefeitura,Centro,Centro,Centro,Centro,Centro


### 1.3.2. Tamanho dos dados

In [132]:
tamanho_dos_dados(df1_subprefeituras)

Colunas: 2
Linhas: 164


In [133]:
qtd_total_unicos_por_coluna(df1_subprefeituras)

Quantidade de valores unicos em nome: 164
Quantidade de valores unicos em subprefeitura: 9


### 1.3.3. Verificar Nulos

In [134]:
df1_subprefeituras.isna().sum()

nome             0
subprefeitura    0
dtype: int64

### 1.3.4. Verificar tipos dos dados

In [135]:
df1_subprefeituras.dtypes

nome             object
subprefeitura    object
dtype: object

### 1.3.5. Verifica dados duplicados

In [136]:
df1_subprefeituras.duplicated().sum()

0

### 1.3.5. Tratamento Inicial

In [137]:
df1_subprefeituras['subprefeitura'] = df1_subprefeituras['subprefeitura'].str.upper()
df1_subprefeituras['nome'] = df1_subprefeituras['nome'].str.upper()

In [138]:
df1_subprefeituras.head().T

,0,1,2,3,4
nome,SANTO CRISTO,GAMBOA,SAÚDE,CAJU,CENTRO
subprefeitura,CENTRO,CENTRO,CENTRO,CENTRO,CENTRO


In [139]:
df1_subprefeituras = remove_acentos(df1_subprefeituras)

### 1.3.6. Salvar dados

In [140]:
salvar_dados_csv(df1_subprefeituras, 'df1_subprefeitura_descricao_dados.csv')

# 2.0. Feature Engineering

In [141]:
df2_escolas = df1_escolas.copy()
df2_material_didatico = df1_material_didatico.copy()
df2_subprefeituras = df1_subprefeituras.copy()

In [142]:
df2_escolas

,id,escolas_postos,bairro,endereço,lat,lon
0,178,CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL,CAJU,RUAA CARLOS SEIDL S/NO,-22.88089,-43.22533
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,RUAA EBANO 187,-22.88957,-43.23620
2,483,EM CELESTINO SILVA,CENTRO,"RUA DO LAVRADIO, 56",-22.90929,-43.18358
3,476,ESCOLA MUNICIPAL FLORIANO PEIXOTO,SAO CRISTOVAO,PRACA ARGENTINA 20,-22.89763,-43.22746
4,132,EM PEREIRA PASSOS,RIO COMPRIDO,PRACA CONDESSA PAULO DE FRONTIN 45,-22.92441,-43.20858
...,...,...,...,...,...,...
146,210,EM DEBORA MENDES DE MORAES,PEDRA DE GUARATIBA,RUAA BELCHIOR DA FONSECA 643,-23.00142,-43.63876
147,474,EM ENGENHEIRO GASTAO RANGEL,GUARATIBA,ESTRADA DO MAGARCA 9.183,-22.98046,-43.64354
148,301,EM JONATAS SERRANO,GUARATIBA,"ESTRADA DO MATO ALTO, S/NO",-22.95316,-43.57741
149,215,E.M. NARCISA AMALIA,ILHA DE GUARATIBA,ESTRADA TEODORETO DE CAMARGO S/N.O,-23.00908,-43.53758


In [143]:
df2_material_didatico

,id,quantidade
0,178,20.0
1,634,121.0
2,483,220.0
3,476,190.0
4,132,67.0
...,...,...
146,474,320.0
147,301,335.0
148,215,342.0
149,606,521.0


In [144]:
df2_subprefeituras

,nome,subprefeitura
0,SANTO CRISTO,CENTRO
1,GAMBOA,CENTRO
2,SAUDE,CENTRO
3,CAJU,CENTRO
4,CENTRO,CENTRO
...,...,...
159,SANTA CRUZ,ZONA OESTE
160,GUARATIBA,ZONA OESTE
161,BARRA DE GUARATIBA,ZONA OESTE
162,PEDRA DE GUARATIBA,ZONA OESTE


## 2.1. Merge dos dados

In [145]:
df2_merged = pd.merge(df2_escolas, df2_subprefeituras, left_on='bairro', right_on='nome', how='left')

In [146]:
df2_merged.sample(5).T

,46,6,99,20,107
id,388,055,175,035,477
escolas_postos,EM CARDEAL CAMARA,EM SANTA CATARINA,E.M. NACOES UNIDAS,EM SENADOR CORREA,EM PAULO MARANHAO
bairro,PARADA DE LUCAS,SANTA TERESA,BANGU,LARANJEIRAS,REALENGO
endereço,"RUA ANAMA, 98",RUAA EDUARDO SANTOS 38,RUAA DOS LIMADORES S/NDEG,RUA SEN. CORREA,RUAA DO GOVERNO 866
lat,-22.81527,-22.91554,-22.88815,-22.93399,-22.89299
lon,-43.30357,-43.19159,-43.47629,-43.18101,-43.44122
nome,PARADA DE LUCAS,SANTA TERESA,BANGU,LARANJEIRAS,REALENGO
subprefeitura,ZONA NORTE,CENTRO,GRANDE BANGU,ZONA SUL,GRANDE BANGU


In [147]:
df2_merged = pd.merge(df2_merged, df2_material_didatico, left_on='id', right_on='id', how='inner')

In [148]:
df2_merged.sample(5).T

,95,99,40,7,63
id,466,080,157,490,140
escolas_postos,ESCOLA MUNICIPAL NELSON RODRIGUES,E.M. JOSE MAURO DE VASCONCELOS,E.M. SARMIENTO,E.M TIA CIATA,EM PEDRO LESSA
bairro,TAQUARA,BANGU,ENGENHO NOVO,CENTRO,BONSUCESSO
endereço,RUAA FREI LUIZ ALEVATO S/NDEG,RUAA MARLIERIA S/NO,RUAA VINTE QUATRO DE MAIO 931,AVENIDA PRESIDENTE VARGAS S/NO,"RUAA ADAIL, 49"
lat,-22.93533,-22.85044,-22.90303,-22.90712,-22.86047
lon,-43.36973,-43.46686,-43.26788,-43.19507,-43.25428
nome,TAQUARA,BANGU,ENGENHO NOVO,CENTRO,BONSUCESSO
subprefeitura,JACAREPAGUA,GRANDE BANGU,ZONA NORTE,CENTRO,ZONA NORTE
quantidade,279.0,599.0,350.0,578.0,153.0


## 2.2. Tratamento de Nulos

In [149]:
df2_merged.isna().sum()

id                0
escolas_postos    0
bairro            0
endereço          0
lat               0
lon               0
nome              9
subprefeitura     9
quantidade        0
dtype: int64

In [150]:
df2_merged[df2_merged['subprefeitura'].isna()].sort_values(by='bairro')

,id,escolas_postos,bairro,endereço,lat,lon,nome,subprefeitura,quantidade
113,262,CIEP HERIVELTO MARTINS,AUGUSTO VASCONCELOS,"PRACA SANTO ANASTACIO, S/N",-22.88649,-43.52880,NaN,NaN,221.0
90,550,EM 25 DE ABRIL,FREGUESIA JPA,RUAA MAMORE 78,-22.93816,-43.34258,NaN,NaN,128.0
94,492,EM MENEZES CORTES,FREGUESIA JPA,PRACA JOSE ALVES DE AZEVEDO 43,-22.94266,-43.33897,NaN,NaN,342.0
29,060,ESCOLA MUNICIPAL BARAO HOMEM DE MELO,MARACANA/ TIJUCA,"RUA ALM. JOAO CANDIDO BRASIL, 352",-22.91683,-43.23692,NaN,NaN,342.0
28,280,ESCOLA MUNICIPAL GENERAL HUMBERTO DE SOUZA MELLO,MARACANA/ VILA ISABEL,"RUA OITO DE DEZEMBRO, 275",-22.90982,-43.23939,NaN,NaN,491.0
131,068,EM WALQUIR PEREIRA,NOVA SEPETIBA,ESTRADA DE SEPETIBA S/N BLOCO 1,-22.95601,-43.68675,NaN,NaN,135.0
79,412,EM MOZART LAGO,OSWALDO CRUZ,RUAA JOSE CARVALHO SALGADO S/N.O,-22.87221,-43.35033,NaN,NaN,395.0
86,528,CIEP MARGARET MEE,RECREIO,RUAA MINISTRO ALIOMAR BALEEIRO S/NO,-23.01981,-43.48172,NaN,NaN,51.0
87,414,EM CLAUDIO BESSERMAN VIANNA - BUSSUNDA,RIO DAS PEDRAS,ESTRADA DE JACAREPAGUA 3327,-22.97336,-43.32916,NaN,NaN,26.0


In [151]:
# id = 262 | AUGUSTO VASCONCELOS = NaN

# id = 550 | FREGUESIA JPA = FREGUESIA (JACAREPAGUA) | JACAREPAGUA
df2_merged.loc[df2_merged['id'] == '550', 'nome'] = 'FREGUESIA (JACAREPAGUA)'
df2_merged.loc[df2_merged['id'] == '550', 'subprefeitura'] = 'JACAREPAGUA'

# id = 492 | FREGUESIA JPA = FREGUESIA (JACAREPAGUA)
df2_merged.loc[df2_merged['id'] == '492', 'nome'] = 'FREGUESIA (JACAREPAGUA)'
df2_merged.loc[df2_merged['id'] == '492', 'subprefeitura']  = 'JACAREPAGUA'

# id = 060 | MARACANA/ TIJUCA = MARACANA
df2_merged.loc[df2_merged['id'] == '060', 'nome'] = 'MARACANA'
df2_merged.loc[df2_merged['id'] == '060', 'subprefeitura'] = 'GRANDE TIJUCA'

# id = 280 | MARACANA/ VILA ISABEL  MARACANA
df2_merged.loc[df2_merged['id'] == '280', 'nome'] = 'MARACANA'
df2_merged.loc[df2_merged['id'] == '280', 'subprefeitura'] = 'GRANDE TIJUCA'

# id = 068 | NOVA SEPETIBA = NaN

# id = 412 | OSWALDO CRUZ = OSVALDO CRUZ
df2_merged.loc[df2_merged['id'] == '412', 'nome'] = 'OSVALDO CRUZ'
df2_merged.loc[df2_merged['id'] == '412', 'subprefeitura']= 'ZONA NORTE'

# id = 528 | RECREIO = RECREIO DOS BANDEIRANTES
df2_merged.loc[df2_merged['id'] == '528', 'nome'] = 'RECREIO DOS BANDEIRANTES'
df2_merged.loc[df2_merged['id'] == '528', 'subprefeitura'] = 'BARRA DA TIJUCA'

# id = 414 | RIO DAS PEDRAS = NaN

In [152]:
df2_merged[df2_merged['subprefeitura'].isna()].sort_values(by='bairro')

,id,escolas_postos,bairro,endereço,lat,lon,nome,subprefeitura,quantidade
113,262,CIEP HERIVELTO MARTINS,AUGUSTO VASCONCELOS,"PRACA SANTO ANASTACIO, S/N",-22.88649,-43.52880,NaN,NaN,221.0
131,068,EM WALQUIR PEREIRA,NOVA SEPETIBA,ESTRADA DE SEPETIBA S/N BLOCO 1,-22.95601,-43.68675,NaN,NaN,135.0
87,414,EM CLAUDIO BESSERMAN VIANNA - BUSSUNDA,RIO DAS PEDRAS,ESTRADA DE JACAREPAGUA 3327,-22.97336,-43.32916,NaN,NaN,26.0


## 2.3. Salvar dados

In [153]:
df2_merged.sample(5).T

,69,65,80,78,12
id,390,069,031,520,450
escolas_postos,E.M. ANDREA FONTES PEIXOTO,ESCOLA MUNICIPAL CARDEAL ARCOVERDE,E.M. PARA,EM BARONESA DE SAAVEDRA,EM ALBERTO BARTH
bairro,PARQUE COLUMBIA,MADUREIRA,ROCHA MIRANDA,REALENGO,FLAMENGO
endereço,RUAA FAUSTO E CASTRO S/NO,RUAA AGOSTINHO BARBALHO 401,AV. DOS ITALIANOS 500,"RUA MAL. BIBIANO COSTALAT, 483-639",AVENIDA OSWALDO CRUZ 124
lat,-22.81631,-22.87608,-22.85196,-22.87185,-22.94071
lon,-43.34015,-43.34337,-43.34949,-43.43243,-43.17647
nome,PARQUE COLUMBIA,MADUREIRA,ROCHA MIRANDA,REALENGO,FLAMENGO
subprefeitura,ZONA NORTE,ZONA NORTE,ZONA NORTE,GRANDE BANGU,ZONA SUL
quantidade,257.0,468.0,115.0,136.0,420.0


In [154]:
salvar_dados_csv(df2_merged, 'df2_feature_engineering.csv')

In [155]:
# Quantidade de material escolar por subprefeitura
df2_merged.groupby('subprefeitura').sum('quantidade')

,lat,lon,quantidade
subprefeitura,,,
BARRA DA TIJUCA,-6.897353e+01,-1.303835e+02,266.0
CENTRO,-2.061402e+02,-3.889166e+02,2352.0
GRANDE BANGU,-3.202665e+02,-6.081691e+02,4321.0
GRANDE TIJUCA,-2.062808e+02,-3.891532e+02,2958.0
ILHAS,-4.562013e+01,-8.639122e+01,741.0
JACAREPAGUA,-2.289714e+07,-4.335306e+07,3289.0
ZONA NORTE,-1.051618e+03,-1.991901e+03,14564.0
ZONA OESTE,-6.420291e+02,-1.221186e+03,9246.0
ZONA SUL,-3.442385e+02,-6.479778e+02,4940.0
